# 可接受任意数量参数的函数

```python
def a(x,*args,**kwargs):
    pass
```
`*args`收集位置参数  
`*kwargs`:收集关键字参数

# 只接受关键之参数的函数

In [3]:
def recv(maxsiez,*,block):
    pass

# 给函数参数增加元信息

In [4]:
def add(x:int,y:int)->int:
    return x+y
help(add)

Help on function add in module __main__:

add(x: int, y: int) -> int



函数注解只存储在函数的`__annotations__`属性中

# 返回多个值得函数

In [5]:
def myfun():
    return 1,2,3

可以使用`,`来生成一个元组

# 有默认参数的函数

In [7]:
def spam(a,b=42):
    pass

In [11]:
_no_value = object()
# 同一性测试
def spam(a,b=_no_value):
    if b is _no_value:
        print('b no value')

In [10]:
spam(1),spam(1,None)

b no value


(None, None)

# 定义匿名或内联函数

In [13]:
names = ['David Beazley', 'Brian Jones',
       'Raymond Hettinger', 'Ned Batchelder']
sorted(names, key=lambda name: name.split()[-1].lower())

['Ned Batchelder', 'David Beazley', 'Raymond Hettinger', 'Brian Jones']

# 匿名函数捕获变量值

In [14]:
x = 10
a = lambda y:x +y
x = 20 
b = lambda y:x+y
a(10),b(10)

(30, 30)

`lambda`表达式中的`x`是一个自由变量，在运行时绑定值  
通过使用函数默认值参数形式，`lambda`函数在定义时就能绑定到值。

In [16]:
funcs = [lambda x:x+n for n in range(5)]
for f in funcs:
    print(f(0))

4
4
4
4
4


In [17]:
funcs = [lambda x,n=n:x+n for n in range(5)]
for f in funcs:
    print(f(0))

0
1
2
3
4


# 减少可调用对象的参数个数

In [18]:
def spam(a,b,c,d):
    print(a,b,c,d)
from functools import partial
s1 = partial(spam,1)
s1(2,3,4)

1 2 3 4


In [19]:
s2 = partial(spam,1,2,d=5)
s2(100)

1 2 100 5


In [21]:
points = [ (1, 2), (3, 4), (5, 6), (7, 8) ]
import math
def distance(p1,p2):
    x1,y1=p1
    x2,y2=p2
    return math.hypot(x2-x1,y2-y1)
pt = (4,3)
points.sort(key=partial(distance,pt))
points

[(3, 4), (1, 2), (5, 6), (7, 8)]

In [29]:
def output_result(result,log=None):
    if log is not None:
        log.debug('Got:%r',result)
def addxy(x,y):
    return x + y

In [ ]:
import logging
from multiprocessing import Pool
from functools import partial
logging.basicConfig(level=logging.DEBUG)
log = logging.getLogger('test')
p = Pool()
p.apply_async(addxy,(3,4),callback=partial(output_result,log=log))
p.close()
p.join()

# 将单方法的类转换为函数

```python
from urllib.request import urlopen

class UrlTemplate:
    def __init__(self, template):
        self.template = template

    def open(self, **kwargs):
        return urlopen(self.template.format_map(kwargs))

# Example use. Download stock data from yahoo
yahoo = UrlTemplate('http://finance.yahoo.com/d/quotes.csv?s={names}&f={fields}')
for line in yahoo.open(names='IBM,AAPL,FB', fields='sl1c1v'):
    print(line.decode('utf-8'))
```

```python
def urltemplate(template):
    def opener(**kwargs):
        return urlopen(template.format_map(kwargs))
    return opener

# Example use
yahoo = urltemplate('http://finance.yahoo.com/d/quotes.csv?s={names}&f={fields}')
for line in yahoo(names='IBM,AAPL,FB', fields='sl1c1v'):
    print(line.decode('utf-8'))
```

# 带额外状态信息的回调函数

In [33]:
def apply_async(func, args, *, callback):
    # Compute the result
    result = func(*args)

    # Invoke the callback with the result
    callback(result)

In [34]:
def print_result(result):
    print('Got:', result)
def add(x, y):
    return x + y
apply_async(add, (2, 3), callback=print_result),apply_async(add, ('hello', 'world'), callback=print_result)

Got: 5
Got: helloworld


(None, None)

In [35]:
class ResultHandler:

    def __init__(self):
        self.sequence = 0

    def handler(self, result):
        self.sequence += 1
        print('[{}] Got: {}'.format(self.sequence, result))

In [36]:
r = ResultHandler()
apply_async(add, (2, 3), callback=r.handler),apply_async(add, ('hello', 'world'), callback=r.handler)

[1] Got: 5
[2] Got: helloworld


(None, None)

In [37]:
def make_handler():
    sequence = 0
    def handler(result):
        nonlocal sequence
        sequence += 1
        print('[{}] Got: {}'.format(sequence, result))
    return handler

In [38]:
handler = make_handler()
apply_async(add, (2, 3), callback=handler),apply_async(add, ('hello', 'world'), callback=handler)

[1] Got: 5
[2] Got: helloworld


(None, None)

In [39]:
def make_handler():
    sequence = 0
    while True:
        result = yield
        sequence += 1
        print('[{}] Got: {}'.format(sequence, result))

In [41]:
handler = make_handler()
next(handler) # Advance to the yield
apply_async(add, (2, 3), callback=handler.send),apply_async(add, ('hello', 'world'), callback=handler.send)

[1] Got: 5
[2] Got: helloworld


(None, None)

# 内联回调函数

In [5]:
def apply_async(func,args,*,callback):
    result = func(*args)
    callback(result)
from queue import Queue
from functools import wraps

class Async:
    def __init__(self,func,args):
        self.func = func
        self.args = args
        
def inlined_async(func):
    @wraps(func)
    def wrapper(*args):
        f = func(*args)
        result_queue = Queue()
        result_queue.put(None)
        while True:
            result = result_queue.get()
            try:
                a = f.send(result)
                apply_async(a.func,a.args,callback=result_queue.put)
            except StopIteration:
                break
    return wrapper

In [6]:
def xadd(x,y):
    return x+y
@inlined_async
def test():
    r = yield Async(xadd,(2,3))
    print(r)
    r = yield Async(xadd,('hello','world'))
    print(r)
    for n in range(10):
        r = yield Async(xadd,(n,n))
        print(r)
    print('GoodBye')

In [8]:
test()

5
helloworld
0
2
4
6
8
10
12
14
16
18
GoodBye


# 访问闭包中定义的变量

In [10]:
def sample():
    n = 0
    # Closure function
    def func():
        print('n=', n)

    # Accessor methods for n
    def get_n():
        return n

    def set_n(value):
        nonlocal n
        n = value

    # Attach as function attributes
    func.get_n = get_n
    func.set_n = set_n
    return func

f = sample()
f(),f.set_n(10),f(),f.get_n()

n= 0
n= 10


(None, None, None, 10)

In [11]:
import sys
class ClosureInstance:
    def __init__(self, locals=None):
        if locals is None:
            locals = sys._getframe(1).f_locals

        # Update instance dictionary with callables
        self.__dict__.update((key,value) for key, value in locals.items()
                            if callable(value) )
    # Redirect special methods
    def __len__(self):
        return self.__dict__['__len__']()

# Example use
def Stack():
    items = []
    def push(item):
        items.append(item)

    def pop():
        return items.pop()

    def __len__():
        return len(items)

    return ClosureInstance()

In [12]:
class Stack2:
    def __init__(self):
        self.items = []

    def push(self, item):
        self.items.append(item)

    def pop(self):
        return self.items.pop()

    def __len__(self):
        return len(self.items)

In [24]:
from timeit import timeit
s = Stack()
timeit('s.push(1);s.pop()', 'from __main__ import s')

1.5869259789997159

In [25]:
s = Stack2()
timeit('s.push(1);s.pop()', 'from __main__ import s')

1.5336146429999644